In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [16]:
df = pd.read_csv("train.csv",index_col=0)

In [24]:
from sklearn.metrics import make_scorer
sample = pd.read_csv("sample_submission.csv",index_col=0)
test = pd.read_csv("test.csv",index_col=0)
sample
def in_top_three_scoring(y_true, y_proba):
    top_3_indices = np.flip(np.argsort(y_proba, axis=1)[:, -3:], axis=1)
    accurate_counts = 0
    for i, val in enumerate(y_true):
        if val in top_3_indices[i]:
            accurate_counts += 1
    return accurate_counts / len(y_true)
top_three_scorer = lambda estimator, X, y : in_top_three_scoring(y, estimator.predict_proba(X))
def mapper(x):
    return fert_encoder.classes_[x]
vectorized_mapper = np.vectorize(mapper)
def predict_test_values(estimator):
    y_test_pred = estimator.predict_proba(test)
    y_test_pred_top3 = np.flip(np.argsort(y_test_pred, axis=1)[:, -3:], axis=1)
    y_test_pred_strings = vectorized_mapper(y_test_pred_top3)
    aaa = np.apply_along_axis(" ".join, arr=y_test_pred_strings, axis=1)
    return aaa
fert_encoder = LabelEncoder()
X = df.drop(columns=['Fertilizer Name'])
y=fert_encoder.fit_transform(df['Fertilizer Name'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,random_state=42)
preprocessor = make_column_transformer((StandardScaler(),("Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous")),(OneHotEncoder(),("Soil Type","Crop Type")))

In [15]:
param_grid = {
    'gb__n_estimators': [50, 100, 200],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 5, 7],
    'gb__subsample': [0.8, 1.0]
}

fert_encoder = LabelEncoder()
X = df.drop(columns=['Fertilizer Name'])
y=fert_encoder.fit_transform(df['Fertilizer Name'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,random_state=42)
preprocessor = make_column_transformer((StandardScaler(),("Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous")),(OneHotEncoder(),("Soil Type","Crop Type")))
GB_OHE = Pipeline([('proc',preprocessor),('gb',GradientBoostingClassifier())])
grid_search_gb = RandomizedSearchCV(GB_OHE, param_grid, cv=5, scoring=top_three_scorer, n_jobs=5,verbose=2)
# Use this to test ^ before running full grid search
# grid_search_gb = GridSearchCV(GB_OHE, param_grid, cv=5, scoring=top_three_scorer, n_jobs=5,verbose=2)
grid_search_gb.fit(X,y)



Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 

In [10]:
grid_gb = grid_search_gb.best_estimator_
print(grid_gb.get_params())
# print(grid_search_gb.best_score_)
best_score_gb = make_pipeline(('proc',preprocessor),('gb',GradientBoostingClassifier(verbose=1)))
best_score_gb.set_params(**grid_gb.get_params())
best_score_gb.fit(X_train,y_train)
y_pred = SGD_OHE.predict_proba(X_test)
in_top_three_scoring(y_test,y_pred)
print(y_pred)
print(in_top_three_scoring(y_test,y_pred))
best_score_gb.fit(X,y)
sample['Fertilizer Name'] = predict_test_values(best_score_gb)

{'memory': None, 'steps': [('proc', ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ('Temparature', 'Humidity', 'Moisture',
                                  'Nitrogen', 'Potassium', 'Phosphorous')),
                                ('onehotencoder', OneHotEncoder(),
                                 ('Soil Type', 'Crop Type'))])), ('gb', GradientBoostingClassifier(learning_rate=0.2, max_depth=7))], 'verbose': False, 'proc': ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ('Temparature', 'Humidity', 'Moisture',
                                  'Nitrogen', 'Potassium', 'Phosphorous')),
                                ('onehotencoder', OneHotEncoder(),
                                 ('Soil Type', 'Crop Type'))]), 'gb': GradientBoostingClassifier(learning_rate=0.2, max_depth=7), 'proc__force_int_remainder_cols': True, 'proc__n_jobs': None, 'proc__remainder': 'drop', 'proc__sparse_t

KeyboardInterrupt: 

In [ ]:
from xgboost import XGBClassifier
param_grid = {
    'gb__n_estimators': [50, 100, 200],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 5, 7],
    'gb__subsample': [0.8, 1.0]
}
xgb_pipeline =  Pipeline([('proc',preprocessor),('gb',XGBClassifier())])
grid_search_xgb = GridSearchCV(xgb_pipeline, param_grid, cv=5, scoring=top_three_scorer, n_jobs=5,verbose=2)
grid_search_xgb.fit(X,y)

In [5]:
from xgboost import XGBClassifier
param_grid = {
    'gb__n_estimators': [50, 100, 200],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 5, 7],
    'gb__subsample': [0.8, 1.0]
}
xgb_pipeline =  Pipeline([('proc',preprocessor),('gb',XGBClassifier())])
rand_search_xgb = RandomizedSearchCV(xgb_pipeline, param_grid, cv=5, scoring=top_three_scorer, n_jobs=1,verbose=2)
rand_search_xgb.fit(X,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END gb__learning_rate=0.01, gb__max_depth=5, gb__n_estimators=100, gb__subsample=1.0; total time=  41.9s
[CV] END gb__learning_rate=0.01, gb__max_depth=5, gb__n_estimators=100, gb__subsample=1.0; total time=  32.4s
[CV] END gb__learning_rate=0.01, gb__max_depth=5, gb__n_estimators=100, gb__subsample=1.0; total time=  31.1s
[CV] END gb__learning_rate=0.01, gb__max_depth=5, gb__n_estimators=100, gb__subsample=1.0; total time=  31.1s
[CV] END gb__learning_rate=0.01, gb__max_depth=5, gb__n_estimators=100, gb__subsample=1.0; total time=  30.5s
[CV] END gb__learning_rate=0.1, gb__max_depth=7, gb__n_estimators=50, gb__subsample=1.0; total time=  19.5s
[CV] END gb__learning_rate=0.1, gb__max_depth=7, gb__n_estimators=50, gb__subsample=1.0; total time=  20.3s
[CV] END gb__learning_rate=0.1, gb__max_depth=7, gb__n_estimators=50, gb__subsample=1.0; total time=  21.3s
[CV] END gb__learning_rate=0.1, gb__max_depth=7, gb__n_estimators

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('proc',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ('Temparature',
                                                                                'Humidity',
                                                                                'Moisture',
                                                                                'Nitrogen',
                                                                                'Potassium',
                                                                                'Phosphorous')),
                                                                              ('onehotencoder',
                                                                               OneHotEncoder(),
                                                                               ('Soil '
                                                                                'Type',
                                                                                'Crop '
                                                                                'Type'))])),
                                             ('gb',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=No...
                                                            max_leaves=None,
                                                            min_child_weight=None,
                                                            missing=nan,
                                                            monotone_constraints=None,
                                                            multi_strategy=None,
                                                            n_estimators=None,
                                                            n_jobs=None,
                                                            num_parallel_tree=None, ...))]),
                   n_jobs=1,
                   param_distributions={'gb__learning_rate': [0.01, 0.1, 0.2],
                                        'gb__max_depth': [3, 5, 7],
                                        'gb__n_estimators': [50, 100, 200],
                                        'gb__subsample': [0.8, 1.0]},
                   scoring=<function <lambda> at 0x000001D8A751C180>,
                   verbose=2)

In [6]:
rand_search_xgb.best_params_

{'gb__subsample': 0.8,
 'gb__n_estimators': 200,
 'gb__max_depth': 5,
 'gb__learning_rate': 0.2}

In [9]:
xgb_rand_best = rand_search_xgb.best_estimator_
xgb_rand_best.fit(X_train,y_train)
in_top_three_scoring(y_test,xgb_rand_best.predict_proba(X_test))

0.5193066666666667

In [10]:
xgb_rand_best.fit(X,y)


IndexError: index 150000 is out of bounds for axis 0 with size 150000

In [11]:
in_top_three_scoring(y,xgb_rand_best.predict_proba(X))

0.5633013333333333

In [26]:
out = predict_test_values(xgb_rand_best)
out[1]

'17-17-17 20-20 10-'

In [21]:
y_test_pred = xgb_rand_best.predict_proba(test)
y_test_pred_top3 = np.flip(np.argsort(y_test_pred, axis=1)[:, -3:], axis=1)
y_test_pred_strings = vectorized_mapper(y_test_pred_top3)
aaa = np.apply_along_axis(" ".join, arr=y_test_pred_strings, axis=1)
aaa


array(['28-28 DAP 10-26-26', '17-17-17 20-20 10-', '20-20 10-26-26 14-',
       ..., '14-35-14 DAP Urea', 'DAP 17-17-17 10-26',
       '17-17-17 14-35-14 '], dtype='<U18')

In [59]:
def predict_test_values(estimator):
    y_test_pred = estimator.predict_proba(test)
    y_test_pred_top3 = np.flip(np.argsort(y_test_pred, axis=1)[:, -3:], axis=1)
    y_test_pred_strings = vectorized_mapper(y_test_pred_top3).astype(object)
    aaa=[]
    for val in y_test_pred_strings:
        aaa.append(" ".join(val))
    return aaa

In [61]:
sample['Fertilizer Name'] = predict_test_values(xgb_rand_best)
sample

,Fertilizer Name
id,
750000,28-28 DAP 10-26-26
750001,17-17-17 20-20 10-26-26
750002,20-20 10-26-26 14-35-14
750003,14-35-14 17-17-17 Urea
750004,20-20 10-26-26 28-28
...,...
999995,17-17-17 14-35-14 28-28
999996,14-35-14 10-26-26 17-17-17
999997,14-35-14 DAP Urea


In [63]:
sample.to_csv("attempt3.csv")